# **Análise de Cesta de Compras: Recomendação de Produtos com Apriori**


In [1]:
import pandas as pd
import numpy as np

In [2]:
# Carregar os dados
df = pd.read_csv('datasets/groceries_groceries.csv')

In [3]:
df.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Converter para formato de lista de transações
# Remover NaN e criar uma lista de itens por transação
transactions = []

for i in range(len(df)):
    # Pegar todos os valores da linha, excluindo NaN
    transaction = df.iloc[i].dropna().tolist()
    # Remover a coluna de contagem se existir (primeira coluna parece ser contagem)
    if len(transaction) > 0:
        transactions.append(transaction[1:])  # Pula a primeira coluna (contagem)

In [5]:
transactions[:5]  # Mostrar as primeiras 5 transações

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Usar TransactionEncoder para formato one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [7]:
# Aplicar o algoritmo Apriori
# min_support: suporte mínimo (ex: 0.01 = 1% das transações)
conjuntos_itens_frequentes = apriori(df_encoded, min_support=0.01, use_colnames=True)
conjuntos_itens_frequentes.head()

,support,itemsets
0,0.033452,(UHT-milk)
1,0.017692,(baking powder)
2,0.052466,(beef)
3,0.033249,(berries)
4,0.026029,(beverages)


In [8]:
# Gerar regras de associação
# min_threshold: Reduzido para 0.2 (20%)
regras = association_rules(conjuntos_itens_frequentes, metric="confidence", min_threshold=0.2)

# Ordenar por lift (força da associação)
regras = regras.sort_values('lift', ascending=False)

print("\nRegras de associação encontradas:")
print(regras.head(10))


Regras de associação encontradas:
                            antecedents                     consequents  \
156    (other vegetables, citrus fruit)               (root vegetables)   
207          (yogurt, other vegetables)            (whipped/sour cream)   
184  (other vegetables, tropical fruit)               (root vegetables)   
2                                (beef)               (root vegetables)   
158     (citrus fruit, root vegetables)              (other vegetables)   
186   (tropical fruit, root vegetables)              (other vegetables)   
187      (other vegetables, whole milk)               (root vegetables)   
190                   (root vegetables)  (other vegetables, whole milk)   
155                            (butter)  (other vegetables, whole milk)   
163                  (curd, whole milk)                        (yogurt)   

     antecedent support  consequent support   support  confidence      lift  \
156            0.028876            0.108998  0.010371    0.3

# Conclusão

Após a aplicação do algoritmo Apriori com os parâmetros ajustados (`min_support=0.01` e `min_threshold=0.2`), o modelo conseguiu identificar diversas regras de associação relevantes que estavam anteriormente ocultas devido a parâmetros muito restritivos.

### **Pergunta: Quais foram as regras de associação mais relevantes?**

A relevância é melhor medida pela coluna **`lift`** (elevação), que indica o quão mais provável é a compra de um item A dado que um item B foi comprado. As regras mais fortes identificadas foram:

### Regras Mais Relevantes (por Lift)

1. **(outros vegetais + frutas cítricas) → raízes/tubérculos**  
   Lift: 3.29 | Confiança: 35.9%  
   *Clientes que compram esses itens são 3.29x mais propensos a comprar raízes/tubérculos.*

2. **(outros vegetais + iogurte) → creme de leite**  
   Lift: 3.26 | Confiança: 23.4%  
   *Clientes que compram esses itens são 3.26x mais propensos a comprar creme de leite.*

3. **(outros vegetais + frutas tropicais) → raízes/tubérculos**  
   Lift: 3.14 | Confiança: 34.2%  
   *Clientes que compram esses itens são 3.14x mais propensos a comprar raízes/tubérculos.*

4. **(carne bovina) → raízes/tubérculos**  
   Lift: 3.04 | Confiança: 33.1%  
   *Clientes que compram carne são 3.04x mais propensos a comprar raízes/tubérculos.*


### **Pergunta: Como elas podem ser aplicadas?**

**Layout da loja:** Posicionar raízes/tubérculos próximo a carnes e frutas; creme de leite próximo a iogurtes e vegetais.

**Promoções:** "Compre carne e ganhe 15% em raízes" ou "Kit Sobremesa" (iogurte + creme de leite).

**Recomendação online:** Sugerir raízes/tubérculos quando o cliente adicionar outros vegetais + frutas cítricas ao carrinho.